Encode with the scaled data


In [ ]:
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl

In [22]:
import torch
import numpy as np
import torch.nn as nn
from sklearn.decomposition import PCA

# import torch_xla
# import torch_xla.core.xla_model as xm
# import torch_xla.distributed.parallel_loader as pl
# import torch_xla.distributed.xla_multiprocessing as xmp

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
unscaled = np.load('/content/drive/MyDrive/Autoencoding/Unscaled_data_sorted.npy')
train_dataset = unscaled[unscaled[:,0] < 2.0] ## taking the distence of relevance

pca_whiten = PCA(whiten=True)

train_dataset = pca_whiten.fit_transform(train_dataset)

train_dataset = torch.from_numpy(train_dataset)

In [25]:
class Swish(nn.Module):
    def __init__(self):
        super(Swish, self).__init__()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return x * self.sigmoid(x)

def lin_block(in_dim,out_dim,bias=True):
    layers = [nn.Linear(in_dim,out_dim,bias= True),
              #nn.BatchNorm2d(out_dim),
              Swish()]
    return nn.Sequential(*layers)

In [26]:
hidden_layers = [2,64,128,256]
class Encoder(nn.Module):
      def __init__(self,hidden_layers):
          super(Encoder,self).__init__()
          self.lin1 = lin_block(hidden_layers[0],hidden_layers[1])         
          self.lin2 = lin_block(hidden_layers[1],hidden_layers[2])
          self.lin3 = lin_block(hidden_layers[2],hidden_layers[3]) 
        
          self.res1 = nn.Sequential(lin_block(hidden_layers[3], hidden_layers[3]),  
                                  lin_block(hidden_layers[3], hidden_layers[3]))
        
          #Encode2
          self.lin4 = lin_block(hidden_layers[3],hidden_layers[2])         
          self.lin5 = lin_block(hidden_layers[2],hidden_layers[1])
        
          self.res2 = nn.Sequential(lin_block(hidden_layers[1], hidden_layers[1]),  
                                  lin_block(hidden_layers[1], hidden_layers[1]))
          self.lin6 = nn.Linear(hidden_layers[1],hidden_layers[0])

      def forward(self,x):
          out = self.lin1(x.float())
          out = self.lin2(out)
          out = self.lin3(out)
          out = self.res1(out)+out
          out = self.lin4(out)
          out = self.lin5(out)
          out = self.res2(out)+out
          out = self.lin6(out)
          return out

class Decoder(nn.Module):
      def __init__(self,hidden_layers):
          super(Decoder,self).__init__()
          self.lin7 = lin_block(hidden_layers[0],hidden_layers[1])         
          self.lin8 = lin_block(hidden_layers[1],hidden_layers[2])
          self.lin9 = lin_block(hidden_layers[2],hidden_layers[3]) 
          
          self.res3 = nn.Sequential(lin_block(hidden_layers[3], hidden_layers[3]),  
                                    lin_block(hidden_layers[3], hidden_layers[3]))
          
          #Decode2
          self.lin10 = lin_block(hidden_layers[3],hidden_layers[2])         
          self.lin11 = lin_block(hidden_layers[2],hidden_layers[1])
          
          self.res4 = nn.Sequential(lin_block(hidden_layers[1], hidden_layers[1]),  
                                    lin_block(hidden_layers[1], hidden_layers[1]))
          self.lin12 = nn.Linear(hidden_layers[1],hidden_layers[0])  

      def forward(self,x):
          out = self.lin7(x.float())
          out = self.lin8(out)
          out = self.lin9(out)
          out = self.res3(out)+out
          out = self.lin10(out)
          out = self.lin11(out)
          out = self.res4(out)+out
          out = self.lin12(out)
          return out

## Add noise to the latent distribution
# change the scale parameter for optimized result.

# def add_noise(latent_z):
#     scale = 1
#     z_mu = latent_z
#     z_log_var = latent_z
#     eps = scale * torch.randn(*z_mu.size()).type_as(z_mu)
#     sampled_z =   torch.exp(z_log_var / 2.) * eps + z_mu
#     return sampled_z

class VDE(nn.Module):
      def __init__(self,*FLAGS):
          super(VDE,self).__init__()
          self.encoder = Encoder(hidden_layers)
          self.decoder = Decoder(hidden_layers)
          #self.add_noise = add_noise()

      def add_noise(self,latent_z):
          scale = 1
          z_mu = latent_z
          z_log_var = latent_z
          eps = scale * torch.randn(*z_mu.size()).type_as(z_mu)
          sampled_z =   torch.exp(z_log_var / 2.) * eps + z_mu
          return sampled_z
          
      def forward(self,x):
          encoded = self.encoder(x)
          z_t = self.add_noise(encoded)
          decoded = self.decoder(z_t)
          return decoded,z_t

      def recons_kl_loss(self,x_decoded,x_shifted,latent_z):
          loss_fn = nn.MSELoss()
          loss = loss_fn(x_decoded,x_shifted)
          z_mu = latent_z
          z_log_var = latent_z
          kl_loss = -0.5* torch.mean(1 + z_log_var -z_mu**2 - torch.exp(z_log_var))
          return loss + kl_loss

      def _corr(self, x, y):
        x = x.view(-1)
        y = y.view(-1)
        mean_x = torch.mean(x)
        mean_y = torch.mean(y)
        xm = x.sub(mean_x.expand_as(x))
        ym = y.sub(mean_y.expand_as(y))
        r_num = xm.dot(ym)
        r_den = torch.norm(xm, 2) * torch.norm(ym, 2)
        r_val = r_num / r_den
        return r_val

      def compute_loss(self,x,x_shifted):
          x_decoded,z_t = self(x)
          latent_unsampled = self.encoder(x)
          auto_correlation_loss = 0
          kl_recons_loss = self.recons_kl_loss(x_decoded,x_shifted,latent_unsampled)
          _,z_tau = self(x_shifted)
          auto_correlation_loss = (1-self._corr(z_t,z_tau))
          return auto_correlation_loss + kl_recons_loss, auto_correlation_loss, kl_recons_loss          

In [27]:
device=torch.device('cuda')
VDE = VDE().to(device)

In [28]:
VDE.load_state_dict(torch.load('/content/drive/MyDrive/VDE_with_scaling/scale_0.1/VDE_state_dict.pt'))

<All keys matched successfully>

In [ ]:
#data = train_dataset[:]

In [ ]:
269*6

1614

In [29]:
x= int(4022018/1614)
print(x)

2491


In [ ]:
import gc
gc.collect()

338

In [ ]:
from tqdm.notebook  import tqdm

In [ ]:
who_ls

['Decoder',
 'Encoder',
 'PCA',
 'Swish',
 'VDE',
 'device',
 'drive',
 'gc',
 'hidden_layers',
 'lin_block',
 'nn',
 'np',
 'pca_whiten',
 'torch',
 'tqdm',
 'train_dataset',
 'unscaled',
 'x']

In [ ]:
#lat1 = make_2_dim(encoder.encode(make_4_dim(train_dataset.to(device))))

In [30]:
%%time
#latent = []
for i in tqdm(range(1,1615)):
    lat1 = VDE.encoder((train_dataset[(i-1)*x:(i*x)].to(device)))
    #z_t = VDE.add_noise(train_dataset[(i-1)*x:(i*x)].to(device))
    torch.save(lat1,'lat'+str(i)+'.pt')
    #latent.append(lat1)
    del lat1 
    gc.collect()


CPU times: user 1min 40s, sys: 679 ms, total: 1min 40s
Wall time: 1min 40s


In [31]:
import glob

In [32]:
file = sorted(glob.glob('lat*'))

In [33]:
file

['lat1.pt',
 'lat10.pt',
 'lat100.pt',
 'lat1000.pt',
 'lat1001.pt',
 'lat1002.pt',
 'lat1003.pt',
 'lat1004.pt',
 'lat1005.pt',
 'lat1006.pt',
 'lat1007.pt',
 'lat1008.pt',
 'lat1009.pt',
 'lat101.pt',
 'lat1010.pt',
 'lat1011.pt',
 'lat1012.pt',
 'lat1013.pt',
 'lat1014.pt',
 'lat1015.pt',
 'lat1016.pt',
 'lat1017.pt',
 'lat1018.pt',
 'lat1019.pt',
 'lat102.pt',
 'lat1020.pt',
 'lat1021.pt',
 'lat1022.pt',
 'lat1023.pt',
 'lat1024.pt',
 'lat1025.pt',
 'lat1026.pt',
 'lat1027.pt',
 'lat1028.pt',
 'lat1029.pt',
 'lat103.pt',
 'lat1030.pt',
 'lat1031.pt',
 'lat1032.pt',
 'lat1033.pt',
 'lat1034.pt',
 'lat1035.pt',
 'lat1036.pt',
 'lat1037.pt',
 'lat1038.pt',
 'lat1039.pt',
 'lat104.pt',
 'lat1040.pt',
 'lat1041.pt',
 'lat1042.pt',
 'lat1043.pt',
 'lat1044.pt',
 'lat1045.pt',
 'lat1046.pt',
 'lat1047.pt',
 'lat1048.pt',
 'lat1049.pt',
 'lat105.pt',
 'lat1050.pt',
 'lat1051.pt',
 'lat1052.pt',
 'lat1053.pt',
 'lat1054.pt',
 'lat1055.pt',
 'lat1056.pt',
 'lat1057.pt',
 'lat1058.pt',
 'lat1

In [34]:
X = []
for f in file:
    X.append(torch.load(f))

X1 = torch.tensor(torch.cat(X))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [35]:
rm -r lat*

In [36]:
torch.save(X1,'latent_not_sampled.pt')

In [37]:
X1.size()

torch.Size([3226386, 2])

In [38]:
#import pyemma

In [39]:
!mv latent_not_sampled.pt /content/drive/MyDrive/VDE_with_scaling/scale_0.1